In [1]:
import polars as pl

In [7]:
(pitching := 
  pl.read_csv("./data/csvFilesLahman/Pitching.csv")
)

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""zuverge01""",1958,1,"""BAL""","""AL""",2,2,45,0,0,0,7,207,74,26,4,17,22,0.28,3.39,3,2,6,0,294,23,29,null,null,null
"""zuverge01""",1959,1,"""BAL""","""AL""",0,1,6,0,0,0,0,39,15,6,1,6,1,0.3,4.15,0,1,0,0,55,1,7,null,null,null
"""zychto01""",2015,1,"""SEA""","""AL""",0,0,13,1,0,0,0,55,17,5,1,3,24,0.239,2.45,0,1,2,0,76,4,6,0,0,0


In [11]:
league_wide_ERA = (
    pitching
    .filter(pl.col("yearID") >= 1946)
    .group_by("yearID", "lgID")
    .agg(total_runs_allowed = pl.sum("R"),
         total_innings_pitched = pl.sum("IPouts"),
         )
    .with_columns([
        (pl.col("total_runs_allowed")/pl.col("total_innings_pitched")*27),
         pl.when(pl.col("yearID") >= 1973)
           .then(pl.lit("DH"))
           .otherwise(pl.lit("NO DH"))
           .alias("DH Rule")
        ])
    .drop("total_innings_pitched", "total_runs_allowed")
              ) 
league_wide_ERA.write_csv("data/league_wide_ERA.csv")

In [12]:
league_wide_ERA.head()
 

yearID,lgID,DH Rule
i64,str,str
2013,"""NL""","""DH"""
2024,"""NL""","""DH"""
2007,"""AL""","""DH"""
1949,"""NL""","""NO DH"""
1989,"""AL""","""DH"""


In [14]:
#Batting data
(batting:=
 pl.read_csv('./data/csvFilesLahman/Batting.csv')
)
batting.head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [24]:
league_wide_SLG = (
    batting
    .filter(pl.col("yearID") >= 1946)
    .group_by(["yearID", "lgID"])
    .agg([ pl.sum("H").alias("total_hits"),
           pl.sum("2B").alias("total_2base"),
           pl.sum("3B").alias("total_3base"),
           pl.sum("HR").alias("total_Hrun"),
           pl.sum("AB").alias("total_Abat")
                              ])
    .with_columns([
        (
            (
                (pl.col("total_hits") - pl.col("total_2base") - pl.col("total_3base") - pl.col("total_Hrun"))
                + 2*pl.col("total_2base") + 3*pl.col("total_3base") + 4*pl.col("total_Hrun")
            )
            / pl.col("total_Abat")
        ), 
        pl.when(
            (pl.col("lgID") == "AL") & (pl.col("yearID") >= 1946)
        ).then(pl.lit("DH"))
        .when(
            (pl.col("lgID") == "NL") & (pl.col("yearID") >= 1973)
        ).then(pl.lit("DH"))
        .otherwise(pl.lit("NO DH"))
        .alias("DH Rule")
    ])
    .drop(["total_hits", "total_2base", "total_3base", "total_Hrun", "total_Abat"])
    
)
league_wide_ERA.write_csv("data/league_wide_SLG.csv")


In [25]:
league_wide_SLG.head()

yearID,lgID,DH Rule
i64,str,str
2005,"""AL""","""DH"""
1981,"""NL""","""DH"""
1990,"""AL""","""DH"""
2007,"""NL""","""DH"""
1971,"""NL""","""NO DH"""
